In [1]:
# 누락데이터 처리
import pandas as pd
import seaborn as sns

In [2]:
df = sns.load_dataset('titanic')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [9]:
# deck 컬럼의 값들의 갯수를 세어 봄 df.value_counts(): 유니크한 값들의 갯수
print(df['deck'].value_counts())
print()
deck_cnt = df['deck'].value_counts(dropna=False) # Nan 데이터 계산하기
print(deck_cnt)
print()

# isnull() 메서드로 누락된 데이터 찾기, null 이면 True, 값이 있으면 False 반환
print(df['deck'].head().isnull())

# isnull()과 sum() 함수를 이용하여 누락된 데이터 갯수 찾기
print(df['deck'].isnull().sum())
df.isnull().sum(axis=0)  # age, deck, embarked, embark_town -> null data 포함

C    59
B    47
D    33
E    32
A    15
F    13
G     4
Name: deck, dtype: int64

NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64

0     True
1    False
2     True
3    False
4     True
Name: deck, dtype: bool
688


survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [ ]:
# 반복문으로 Nan 데이터 개수 계산하기
nan_data = df.isnull()
for col in nan_data.columns:
    nan_cnt = nan_data[col].value_counts()  # 각 열의 Nan 개수 파악

    try:
        print(col, ':', nan_cnt[True])  # Nan 값이 존재하면 개수를 출력
    except:
        print(col, ':', 0)  # Nan 값이 없으면 0를 출력

# Nan 데이터 확인 -> 어떻게 처리 할 건지?
# Nan 데이터가 있는 컬럼을 삭제?, Nan 데이터가 있는 행을 삭제?

In [ ]:
# df.dropna() : axis=1 열을 삭제, axis=0 행을 삭제
# dropna(thresh=500) : Nan이 존재하는 컬럼을 삭제, 
# 갯수가 500 이상인 컬럼만 삭제
df_thresh = df.dropna(axis=1, thresh=500) # axis='columns'
df_thresh.info()

# age에 Nan이 존재하는 행을 삭제
df_age = df.dropna(subset=['age'], how='any', axis='index')
df_age.info()

In [30]:
sr = pd.Series([1,2,3,4,5])
sr_mask = [True, False, True, True, False]
sr[sr_mask].sum()  # 1 + 3 + 4 => 8

8

In [32]:
df.columns[ df.isnull().sum() > 0 ] # Nan이 존재하는 컬럼명만 추출

Index(['age', 'embarked', 'deck', 'embark_town'], dtype='object')

In [33]:
# 컬럼 리스트 : Nan이 존재하는 컬럼 리스트
nan_col = [ df.isnull().sum()>0 ]
print(nan_col, df.columns)
# Nan이 존재하는 모든 행을 삭제 
# nan_col_names = list(df.columns[ nan_col ])
nan_col_names = list(df.columns[ df.isnull().sum()>0 ])
# nan_col_names
df_nan = df.dropna(subset = nan_col_names, how='any', axis=0)
df_nan.info()

[survived       False
pclass         False
sex            False
age             True
sibsp          False
parch          False
fare           False
embarked        True
class          False
who            False
adult_male     False
deck            True
embark_town     True
alive          False
alone          False
dtype: bool] Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 182 entries, 1 to 889
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     182 non-null    int64   
 1   pclass       182 non-null    int64   
 2   sex          182 non-null    object  
 3   age          182 non-null    float64 
 4   sibsp        182 non-null    int64   
 5   parch        182 non-null    int64   
 6   fare         182 non-null   

In [ ]:
# 누락 데이터 치환 : df.fillna(값 또는 method=ffill/ bfill, inplace=True)
print(df['age'].isnull().sum())  # Nan 존재 함

# Nan이 입력된 age를 평균 나이로 치환
df_age = df.copy()
df_age['age'].fillna(df_age['age'].mean(axis=0), inplace=True)
df_age['age'].isnull().sum()
print(df['age'].head(10), df_age['age'].head(10))

In [ ]:
# 누락 데이터 치환
# embark_town 825~ 831 행 출력
# 가장 빈번하게 나오는 값으로 치환
df.embark_town[825:832]
df['embark_town'].value_counts().idxmax()  # 가장 빈번하게 발생하는 인덱스명
df_em = df.copy()
df_em['embark_town'].fillna(df_em['embark_town'].value_counts().idxmax(), 
                           inplace=True)
df_em.embark_town[825:832]

In [56]:
# 누락 데이터 치환 : 이전 데이터로 치환
df_me = df.copy()
df_me['embark_town'].fillna(method='ffill', inplace=True)

print(df_me.embark_town[[828,829]],'\n',df.embark_town[[828,829]])

828    Queenstown
829    Queenstown
Name: embark_town, dtype: object 
 828    Queenstown
829           NaN
Name: embark_town, dtype: object


In [65]:
#  중복 데이터 처리 : df.duplicated() -> 중복 여부 확인
df1 = pd.DataFrame( {'c1': ['a', 'a', 'b', 'a', 'b'],
                    'c2': [1, 1, 1, 2, 2],
                    'c3': [1, 1, 2, 2, 2]})
print(df1)
df_dup = df1.duplicated()
col_dup = df1['c1'].duplicated()

# 중복된 데이터를 제거 , 행을 제거
df2 = df1.drop_duplicates()
df2

# 중복된 컬럼을 제거
df3 = df1.drop_duplicates(subset=['c2','c3'])
df3

  c1  c2  c3
0  a   1   1
1  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


,c1,c2,c3
0,a,1,1
2,b,1,2
3,a,2,2


In [19]:
import time
import seaborn as sns

df = sns.load_dataset('titanic')

In [28]:
# titanic 에서 age, fare, class, alive 컬럼만 가져와서 df_titanic 로 저장한 후
df_titanic = df.loc[:, ['age','fare','class','alive']]
df_titanic.head()

,age,fare,class,alive
0,22.0,7.2500,Third,no
1,38.0,71.2833,First,yes
2,26.0,7.9250,Third,yes
3,35.0,53.1000,First,yes
4,35.0,8.0500,Third,no


In [29]:
df_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   age     714 non-null    float64 
 1   fare    891 non-null    float64 
 2   class   891 non-null    category
 3   alive   891 non-null    object  
dtypes: category(1), float64(2), object(1)
memory usage: 22.0+ KB


In [30]:
# Nan 있는 컬럼의 값의 숫자는 평균으로, 문자는 이전 값으로 대체
for col in df_titanic.columns:
    if df_titanic[col].dtype == 'float64':
#         print("float64 " , col)
        df_titanic[col].fillna(df_titanic[col].mean(), inplace=True)
    else:
#         print("not float64 " , col)
        df_titanic[col].fillna(method='ffill', inplace=True)
df_titanic.head(6) 
len(df_titanic)

891

In [31]:
# 중복된 행과 컬럼은 삭제 하세요
df_titanic.drop_duplicates(inplace=True)
len(df_titanic)

746

In [32]:
df_titanic.drop_duplicates(subset=['age'], inplace=True)
len(df_titanic)

89

In [34]:
import pandas as pd

In [45]:
# dataset/auto_mpg.csv
# 데이터 표준화
df = pd.read_csv('dataset/auto-mpg.csv', header=None)

# 컬럼 이름 지정
df.columns = ['mpg','cylinders','displacement','horsepower','weight',
              'acceleration','model year','origin','name'] 

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    float64
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   name          398 non-null    object 
dtypes: float64(4), int64(3), object(2)
memory usage: 28.1+ KB


,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


In [46]:
# 단위 환산 : mpg -> gallon => kg으로 변환
mpg_to_kg = 1.60934 / 3.78541
df['kpl'] = (df['mpg'] * mpg_to_kg).round(2)
df.head(2)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,7.65
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,6.38


In [ ]:
# 자료형 변환 : object -> float으로 변경
df['horsepower'].unique() # '?' 발견 -> Nan으로 변경후 Nan 처리함수 사용 가능

In [48]:
import numpy as np
df['horsepower'].replace('?', np.nan, inplace=True)

# 데이터 타입을 float으로 변경
df['horsepower'] = df['horsepower'].astype('float')

In [55]:
# origin : 숫자로 표기되어 있음 
df.origin.unique()  # 숫자를 category로 변경 : 제조국 이름으로 변경
df.origin.replace({1:'USA', 2:'EU', 3:'JPN'}, inplace=True)
df.origin.unique()

# object를 카테고리로 변경
df.origin = df.origin.astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   mpg           398 non-null    float64 
 1   cylinders     398 non-null    int64   
 2   displacement  398 non-null    float64 
 3   horsepower    392 non-null    float64 
 4   weight        398 non-null    float64 
 5   acceleration  398 non-null    float64 
 6   model year    398 non-null    int64   
 7   origin        398 non-null    category
 8   name          398 non-null    object  
 9   kpl           398 non-null    float64 
dtypes: category(1), float64(6), int64(2), object(1)
memory usage: 28.6+ KB


In [56]:
# 제조 년도 : model year -> 카테고리로 변경
df['model year'].unique()

# 연도 숫자를 category로 변경
df['model year'] = df['model year'].astype('category')
df.info()
df['model year'].unique

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   mpg           398 non-null    float64 
 1   cylinders     398 non-null    int64   
 2   displacement  398 non-null    float64 
 3   horsepower    392 non-null    float64 
 4   weight        398 non-null    float64 
 5   acceleration  398 non-null    float64 
 6   model year    398 non-null    category
 7   origin        398 non-null    category
 8   name          398 non-null    object  
 9   kpl           398 non-null    float64 
dtypes: category(2), float64(6), int64(1), object(1)
memory usage: 26.6+ KB


<bound method Series.unique of 0      70
1      70
2      70
3      70
4      70
       ..
393    82
394    82
395    82
396    82
397    82
Name: model year, Length: 398, dtype: category
Categories (13, int64): [70, 71, 72, 73, ..., 79, 80, 81, 82]>

In [ ]:
# 데이터 전처리
# 1. 누락 데이터 처리 : 제거, 치환 
# 2. 중복 데이터 처리 : 제거(행), 컬럼..
# 3. 자료 표준화
#    1) 단위 환산
#    2) 자료형 변경 : object -> float, object -> category
#       2.1) 변환 불가능한 자료는 Nan으로 치환하고 자료형 변경
#

In [ ]:
#